In [1]:
# ADDS DATA FROM SPECIFIED CSV TO FIRESTORE COLLECTION. ALL FIELDS AS STRINGS.

In [2]:
# https://medium.com/@cbrannen/importing-data-into-firestore-using-python-dce2d6d3cd51
# Use conda environment 'firestore-operations'
# https://code.visualstudio.com/docs/python/environments#_conda-environments
""" 
To ensure the environment is set up well from a shell perspective, 
one option is to use an Anaconda prompt with the activated environment 
to launch VS Code using the code . command. At that point you just 
need to select the interpreter using the Command Palette or by clicking 
on the status bar.
"""

import csv
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore

In [3]:
cred = credentials.Certificate("../secrets/ServiceAccountKey.json")
app = firebase_admin.initialize_app(cred)

store = firestore.client()


In [4]:
file_path = # set path
collection_name = "triviaContent"

In [5]:
def batch_data(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [6]:
data = []
headers = []
with open(file_path, encoding="utf8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            for header in row:
                headers.append(header)
            line_count += 1
        else:
            obj = {}
            for idx, item in enumerate(row):
                obj[headers[idx]] = item
            data.append(obj)
            line_count += 1
    print(f'Processed {line_count} lines.')

for batched_data in batch_data(data, 499):
    batch = store.batch()
    for data_item in batched_data:
        doc_ref = store.collection(collection_name).document()
        batch.set(doc_ref, data_item)
    batch.commit()
    
print('Done')

Processed 61904 lines.
Done
